In [ ]:
import boto3
import os
import subprocess
from botocore.client import Config

# MinIO Configuration
MINIO_ENDPOINT = "http://127.0.0.1:9000"
ACCESS_KEY =  "minioadmin"
SECRET_KEY =  "minioadmin"
BUCKET_NAME = "7dcf1193-4237-48a7-a5f2-4b530b69b1cb"

# Initialize MinIO S3 Client
s3_client = boto3.client(
    "s3",
    endpoint_url=MINIO_ENDPOINT,
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    config=Config(signature_version="s3v4"),
)

# List all objects in 'download_from_stac/' folder
objects = s3_client.list_objects_v2(Bucket=BUCKET_NAME, Prefix="downloaded_from_stac")

if "Contents" in objects:
    for obj in objects["Contents"]:
        file_key = obj["Key"]  # Full path in MinIO

        if file_key.endswith(".tif"):  # Process only TIFF files
            # Extract subfolder and filename
            relative_path = file_key.replace("downloaded_from_stac/", "")  # Remove base path
            subfolder_path = os.path.dirname(relative_path)  # e.g., C3_MX_20240421_2484919101
            filename = os.path.basename(relative_path)  # e.g., BAND1.tif

            # Local temp file paths
            local_tif = f"{filename}"
            local_cog = f"{filename.replace('.tif', '_cog.tif')}"

            # Download the TIFF file
            s3_client.download_file(BUCKET_NAME, file_key, local_tif)
            
            # Convert to COG using GDAL
            cmd = [
                "gdal_translate",
                "-of", "COG",  # Output format
                "-co", "COMPRESS=LZW",  # Compression
                "-co", "BIGTIFF=YES",  # Handle large files
                "-co", "OVERVIEWS=AUTO",  
                local_tif,
                local_cog,
            ]
            subprocess.run(cmd, check=True)

            # Define output key with the same subfolder structure in "cogtiffs/"
            output_key = f"cogtiffs/{subfolder_path}/{filename.replace('.tif', '_cog.tif')}"

            # Upload the COG to MinIO
            s3_client.upload_file(local_cog, BUCKET_NAME, output_key)

            print(f"Converted {file_key} -> {output_key}")

            # Cleanup
            os.remove(local_tif)
            os.remove(local_cog)

else:
    print("No TIFF files found in 'download_from_stac/'.")
